In [6]:
import cv2
import numpy as np
#from google.colab.patches import cv2_imshow
import ipywidgets as widgets
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [7]:
# Load image
doGray = 1

img = cv2.imread('sample2.jpg')

# Convert to grayscale
if (doGray):
    imgToBlur = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    imgtoSharp = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    imgtoBorder = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
else:
    imgToBlur, imgtoBorder, imgtoSharp = img, img, img



## Kernel Blur
este kernel obtiene el valor promedio de todos los pixeles en una matriz de n x n, ese valor promedio genera una difusión de los colores de la imagen creando ese efecto borroso

In [8]:

def apply_blur(kernel_size):
    # kernel_size must be odd and >= 1
    if kernel_size % 2 == 0:
        kernel_size += 1

    kernel = np.ones((kernel_size, kernel_size), np.float32) / (kernel_size**2)

    # Apply blur
    blurred = cv2.filter2D(imgToBlur, -1, kernel)
    
    print(f"Kernel size: {kernel_size}x{kernel_size}")
    combined = np.hstack((imgToBlur, blurred))
    
    combined_rgb = cv2.cvtColor(combined, cv2.COLOR_BGR2RGB)

    plt.imshow(combined_rgb, cmap='gray')
    plt.axis('off')
    plt.show()

# Slider widget (odd numbers only)
slider = widgets.IntSlider(value=3, min=1, max=51, step=2)

widgets.interact(apply_blur, kernel_size=slider);

interactive(children=(IntSlider(value=3, description='kernel_size', max=51, min=1, step=2), Output()), _dom_cl…

## Kernel Sharp
Este kernel realza el contraste entre pixeles en sus bordes, similar al edge detection, pero de una forma más simple, por lo que se centra más en encontrar cambios fuertes de color

In [9]:
def apply_sharpen(intensity):
    # Adjust sharpening strength (center value = 5 + intensity)
    sharp_kernel = np.array([
        [0, -1,  0],
        [-1,  4 + intensity, -1],
        [0, -1,  0]
    ], np.float32)

    sharpened = cv2.filter2D(imgtoSharp, -1, sharp_kernel)

    combined = np.hstack((imgtoSharp, sharpened))

    combined = cv2.cvtColor(combined, cv2.COLOR_BGR2RGB)

    plt.imshow(combined, cmap='gray')
    plt.axis('off')
    plt.show()

slider = widgets.IntSlider(value=1, min=0, max=20, step=1, description='Sharpness')

widgets.interact(apply_sharpen, intensity=slider);


interactive(children=(IntSlider(value=1, description='Sharpness', max=20), Output()), _dom_classes=('widget-in…

## Kernel Edge
Este kernel realza aún más los bordes combinando más de un kernel con el fin de obtener los bordes en todas las direcciones, luego se obtiene el valor mayor de estos kernels con lo que se obtienen los bordes

In [10]:
# Define kernels

kirsch_kernels = [
    np.array([[ 5,  5,  5],
              [-3,  0, -3],
              [-3, -3, -3]], np.float32),  # North

    np.array([[-3,  5,  5],
              [-3,  0,  5],
              [-3, -3, -3]], np.float32),  # Northwest

    np.array([[-3, -3,  5],
              [-3,  0,  5],
              [-3, -3,  5]], np.float32),  # West

    np.array([[-3, -3, -3],
              [-3,  0,  5],
              [-3,  5,  5]], np.float32),  # Southwest

    np.array([[-3, -3, -3],
              [-3,  0, -3],
              [ 5,  5,  5]], np.float32),  # South

    np.array([[-3, -3, -3],
              [ 5,  0, -3],
              [ 5,  5, -3]], np.float32),  # Southeast

    np.array([[ 5, -3, -3],
              [ 5,  0, -3],
              [ 5, -3, -3]], np.float32),  # East

    np.array([[ 5,  5, -3],
              [ 5,  0, -3],
              [-3, -3, -3]], np.float32),  # Northeast
]

def enhance_edges_corners(strength):
    # Scale kernels with intensity
    #lap_k = laplacian_kernel * (1 + strength / 5)
    cor_k =  [k * (strength / 5) for k in kirsch_kernels]

    #edges = cv2.filter2D(imgtoBorder, -1, lap_k)

    # Combine edge and corner maps
    responses = [cv2.filter2D(imgtoBorder, -1, k) for k in cor_k]
    bordered = np.max(responses, axis=0)

    # Convert to 3 channels for side-by-side display

    combined = np.hstack((imgtoBorder, bordered))

    combined = cv2.cvtColor(combined, cv2.COLOR_BGR2RGB)
    
    plt.imshow(combined, cmap='gray')
    plt.axis('off')
    plt.show()
# Slider (0–10)
slider = widgets.IntSlider(value=1, min=0, max=10, step=1, description='Enhance')

widgets.interact(enhance_edges_corners, strength=slider);


interactive(children=(IntSlider(value=1, description='Enhance', max=10), Output()), _dom_classes=('widget-inte…